In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(root="data", train=True,
    download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False,
    download=True, transform=ToTensor())

torch.manual_seed(23)
train_dataloader = DataLoader(training_data, batch_size=64,
  shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64,
  shuffle=True)

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, layers: list, num_classes):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.layers = nn.ModuleList()
    self.input_size = input_size
    for output_size, activation_function in layers:
      self.layers.append(nn.Linear(input_size, output_size))
      input_size = output_size
      self.layers.append(activation_function)
    self.layers.append(nn.Linear(input_size, num_classes))
  def forward(self, x):
    x = self.flatten(x)
    for layer in self.layers:
      x = layer(x)
    return x

torch.manual_seed(23)
C = 10
N,H,W = training_data.data.shape; D = H*W
M1, M2 = 512, 512
model = NeuralNetwork(D, [(M1, nn.ReLU()), (M2, nn.ReLU())], C)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X); loss = loss_fn(pred, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step() # backprop
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"trloss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset); nbatches = len(dataloader)
    teloss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X); teloss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    teloss /= nbatches; correct /= size
    print(f"teacc: {(100*correct):>0.1f}%, teloss: {teloss:>8f} \n")

learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Epoch 1
-------------------------------
trloss: 2.295686  [    0/60000]
trloss: 0.825578  [ 6400/60000]
trloss: 0.363066  [12800/60000]
trloss: 0.406248  [19200/60000]
trloss: 0.479835  [25600/60000]
trloss: 0.422429  [32000/60000]
trloss: 0.255051  [38400/60000]
trloss: 0.336121  [44800/60000]
trloss: 0.478418  [51200/60000]
trloss: 0.327682  [57600/60000]
teacc: 84.5%, teloss: 0.418623 

Epoch 2
-------------------------------
trloss: 0.276260  [    0/60000]
trloss: 0.376784  [ 6400/60000]
trloss: 0.225254  [12800/60000]
trloss: 0.329928  [19200/60000]
trloss: 0.427377  [25600/60000]
trloss: 0.112671  [32000/60000]
trloss: 0.279801  [38400/60000]
trloss: 0.297771  [44800/60000]
trloss: 0.310086  [51200/60000]
trloss: 0.276986  [57600/60000]
teacc: 85.7%, teloss: 0.396044 

Epoch 3
-------------------------------
trloss: 0.397765  [    0/60000]
trloss: 0.260031  [ 6400/60000]
trloss: 0.390639  [12800/